In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import operator

YEARS = 5

def sort_dict_by_values(a, reverse=True):
    sorted_x = sorted(a.items(), key=operator.itemgetter(1), reverse=reverse)
    return sorted_x


s = pd.read_csv('../input/kaggle-winners/Kaggle_Winners.csv')
print('Columns: {}'.format(list(s.columns.values)))
url_norm = []
for url in s['url'].values:
    url_norm.append(url[:-11])
s['url_norm'] = url_norm
print(s['url_norm'])
dates = pd.read_csv('../input/kaggle-winners/Kaggle_Contest_Dates.csv')
url_dates = list(dates['url'].values)

part = s[s['url_norm'] == '']
print(len(part))
print(part)

print(len(set(url_norm)))
print(len(set(url_dates)))
print(len(set(url_norm) & set(url_dates)))
print(set(url_norm) - set(url_dates))
print(set(url_dates) - set(url_norm))

dates['url_norm'] = dates['url']
dates_fix = dates[['url_norm', 'date_end']].copy()
s = s.merge(dates, on='url_norm', how='left')

if YEARS:
    print(len(s))
    s['date_end'] = pd.to_datetime(s['date_end'])
    print(s['date_end'])
    years = YEARS
    limit_date = pd.to_datetime('now').floor('d') - pd.DateOffset(days=years * 365)
    s = s[s['date_end'] > limit_date]
    print('!!!', len(s))


print(s['winner'].value_counts())
unique_winners = s['winner'].unique()

print(s['money'].describe())

res_money = dict()
res_count = dict()
for u in unique_winners:
    part = s[s['winner'] == u].copy()
    part['money'] = part['money'].astype(np.float32)
    part['money_reduced'] = part['money'] / part['team_size']
    res_money[u] = part['money_reduced'].sum()
    res_count[u] = len(part)

res_money = sort_dict_by_values(res_money)
winners_meta = pd.read_csv('../input/kaggle-winners/Kaggle_Nicks_and_Avatars.csv')
res_meta = dict()
for index, row in winners_meta.iterrows():
    res_meta[row['url']] = [row['nick'], row['avatar']]
# print(res_money)

title = 'Kaggle money rating'
if YEARS:
    title += ' (for {} years)'.format(YEARS)
else:
    title += ' (all time)'

In [ ]:
from IPython.display import HTML

html = """
<html>
  <link rel="stylesheet" type="text/css" href="/static/assets/app.css">
  <head>
    <meta name="generator"
    content="HTML Tidy for HTML5 (experimental) for Windows https://github.com/w3c/tidy-html5/tree/c63cc39" />
    <title></title>
  </head>
  <body>
  <a name="rating"></a>
  <h1>{}</h1>
    <div class="sc-jLYMDn fnFWSq">
      <div>
        <div class="leaderboards__item-wrapper leaderboards__item-wrapper--header">
          <div class="leaderboards__rank" style="text-align: center;">Rank</div>
          <div class="leaderboards__avatar">
            <div class="leaderboards__user-label">User</div>
          </div>
          <div class="leaderboards__name"></div>
		  <div class="leaderboards__medals" style="text-align: center;">Prizes num</div>
		  <div class="leaderboards__points" style="text-align: center;">Total prize money ($)</div>
        </div>
      </div>
""".format(title)

rank = 1
for url, money in res_money[:100000]:
    if url.lower() == 'unknown':
        continue
    count = res_count[url]
    if url in res_meta:
        name, avatar = res_meta[url]
    else:
        name = url.split('/')[1]
        avatar = 'https://storage.googleapis.com/kaggle-avatars/images/default-thumb.png'
    html += """
          <div role="button">
            <div class="block-link block-link--bordered">
              <a class="block-link__anchor" href="{}"></a>
              <div class="leaderboards__item-wrapper null">
                <div class="leaderboards__rank" style="text-align: center !important;">{}</div>
                <div class="leaderboards__avatar">
                  <a href="{}" style="z-index: 2; pointer-events: auto;">
                    <img alt="thumnailUrl" src="{}" />
                  </a>
                </div>
                <div class="leaderboards__name">
                  <p>
                    <a href="{}" style="z-index: 2; pointer-events: auto;">{}</a>
                  </p>
                </div>
                <div class="leaderboards__medals" style="text-align: center;">
                    {}
                </div>
                <div class="leaderboards__points" style="text-align: center;"><b>{:.0f}</b></div>
              </div>
            </div>
          </div>
""".format(url, rank, url, avatar, url, name, count, money)
    rank += 1

html += """
	</div>
  </body>
</html>
"""

display(HTML(html))